# Простая классификация по тональности
 

Собираем данные с сайта Кинопоиск: последние 200 положительный, 200 нейтральных и 200 отрицательных отзывов.

In [7]:
import sys
import os
import urllib2
import urllib
from lxml import html
from newspaper import Article
bad_main_url = 'http://www.kinopoisk.ru/reviews/type/comment/status/bad/'
bad_urls = []

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/600.3.18 (KHTML, like Gecko) Version/8.0.3 Safari/600.3.18'
values = {'name' : 'HSE', 'location' : 'Russia', 'language' : 'Nohtyp' }
headers = { 'User-Agent' : user_agent }
data = urllib.urlencode(values)

def get_review_urls(main_url):
    urls = []
    for i in range(1, 21):
        cur_url = main_url + 'period/month/page/'+str(i)+'/#list'
        req = urllib2.Request(cur_url, data, headers)
        response = urllib2.urlopen(req)
        page = response.read()
        code = ''
        for line in page:
            code += line
        tree = html.fromstring(code)
        all_urls = tree.xpath('//a/@href')
        for url in all_urls:
            if 'user' in url and 'comment' in url and '#comm0' not in url:
                if not url in bad_urls:
                    urls.append(url)
    return urls

bad_main_url = 'http://www.kinopoisk.ru/reviews/type/comment/status/bad/'
bad_urls = get_review_urls(bad_main_url)

good_main_url = 'http://www.kinopoisk.ru/reviews/type/comment/status/good/'
good_urls = get_review_urls(good_main_url)

neutral_main_url = 'http://www.kinopoisk.ru/reviews/type/comment/status/neutral/'
neutral_urls = get_review_urls(neutral_main_url)

print len(bad_urls), len(good_urls), len(neutral_urls)


200 200 200


Парсер html страничек с отзывавами. 

In [25]:
import re
from HTMLParser import HTMLParser

class MyHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.data = []
    def handle_data(self, data):
        self.data.append(data)


def getData(text, pattern):
    p = re.compile(pattern, re.DOTALL)
    try:
        m = p.search(text).group()
    except AttributeError:
        return
    else:
        parser = MyHTMLParser()
        parser.feed(m)
        return parser.data

def get_reviews(urls):
    texts = []
    for url in urls:
        comment_url = 'http://www.kinopoisk.ru'+ url
        req = urllib2.Request(comment_url, data, headers)
        response = urllib2.urlopen(req)
        page = response.read()
        code = ''
        text_p = '<span class="_reachbanner_" itemprop="reviewBody">.*?</span>'
        for line in page:
            code += line
        text = getData(code, text_p)
        texts.append(' '.join(text).decode('windows-1251'))
    return texts

bad_reviews = get_reviews(bad_urls)
good_reviews = get_reviews(good_urls)
neutral_reviews = get_reviews(neutral_urls)

print len(bad_reviews)
print len(good_reviews)
print len(neutral_reviews)

200
200
200


Сохранение данных.

In [32]:
data = {}
data['reviews'] = []
for i in bad_reviews: 
    data['reviews'].append({'review':i, 'class':'negative'})
for i in good_reviews: 
    data['reviews'].append({'review':i, 'class':'positive'})
for i in neutral_reviews: 
    data['reviews'].append({'review':i, 'class':'neutral'})

import json
with open('data/kino_data.txt', 'w') as outfile:
    json.dump(data, outfile)

Стандартная предобработка данных.

In [4]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
exclude = set(punctuation + u'0123456789[]—«»')
from pymorphy2 import MorphAnalyzer

def preprocess(text):
    buf = ''.join(ch for ch in text if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(buf.lower())
    lemmatizer = MorphAnalyzer()
    lemmas = []
    for t in tokens[:]:
        if not t in stopwords.words('russian'):
            try:
                lemma = lemmatizer.parse(t)[0].normal_form
            except: 
                lemma = t
#             print t, type(t), lemma
            lemmas.append(lemma)
    return lemmas

Запуск предобработки. 

In [5]:
import json
with open('data/kino_data.txt') as infile:
    data = json.load(infile)
print 'N of reviews:', len(data['reviews'])
texts = []
target = []
for r in data['reviews']:
    text = ' '.join(preprocess(r['review'])).strip()
    texts.append(text)
    target.append(r['class'])
print texts[0]
      

N of reviews: 600
случай весь текст нужный правильно расставлять кавычка понять свой ум автор первый наверное самый важный премьер год фильм который пропустить кой случай триумфальный возвращение алекс пройас большой кино очередной подтверждение безупречный реноме джерард батлера серьёзный придирчивый относиться выбор роль актёр долгожданный пропуск николай костеравальдау общество актёр класс девочка прошлогодний безумный макс нужный счастие перейти поразительный потрясать плюс картина взгляд минус вообще такой произведение задаваться вопрос выйти долгожданный продолжение фильм битва титан гнев титан бог египет весь свой красота давать положительный ответ кульминационный пиршество время битва бог титан монстр коготь заключить именно творение алекс пройас прежде вызволить любимый гнёт один важный египетский шишка молодой юнец требоваться замедлить кадр обязательно поцеловать девушка нужный убегать нога вперёд поцелуй важный древний египет мочь найтись юноша тело райана рейнольдс дева те

150 случайных документов – обучающая выборка, остальные 50 - тестовая.

In [32]:
import random
train_idx = random.sample(range(0,201),  150) + random.sample(range(200, 401),  150) + random.sample(range(400, 600),  150) 
print train_idx

[148, 62, 90, 13, 161, 188, 171, 186, 163, 118, 79, 122, 130, 150, 27, 87, 71, 119, 59, 123, 172, 101, 193, 134, 30, 22, 113, 177, 29, 78, 105, 120, 129, 89, 55, 195, 175, 81, 65, 9, 151, 165, 94, 164, 200, 99, 25, 115, 40, 141, 179, 70, 108, 104, 167, 91, 52, 8, 124, 176, 73, 166, 103, 33, 43, 153, 69, 180, 17, 147, 86, 97, 54, 85, 18, 125, 154, 49, 191, 23, 12, 199, 20, 135, 156, 42, 57, 128, 77, 48, 136, 36, 11, 183, 149, 24, 162, 67, 132, 31, 32, 190, 98, 182, 144, 35, 46, 5, 15, 68, 142, 83, 95, 185, 10, 197, 181, 155, 110, 189, 187, 143, 4, 168, 146, 158, 16, 174, 14, 126, 66, 137, 184, 88, 37, 39, 58, 74, 169, 44, 21, 1, 139, 60, 138, 117, 152, 198, 159, 76, 226, 228, 378, 298, 324, 271, 349, 213, 306, 283, 366, 217, 216, 206, 372, 249, 255, 267, 278, 269, 319, 258, 316, 337, 307, 367, 290, 246, 382, 265, 294, 305, 397, 286, 398, 385, 275, 351, 288, 387, 357, 386, 214, 338, 327, 329, 325, 368, 320, 289, 256, 391, 395, 360, 250, 225, 312, 364, 393, 321, 389, 394, 210, 245, 317, 3

Формирование обучающей выборки. 

In [33]:
train_texts = []
train_target = []
test_texts = []
test_target = []

for t_id in range(len(texts)):
    if t_id in train_idx:
        train_texts.append(texts[t_id])
        train_target.append(target[t_id])
    else:
        test_texts.append(texts[t_id])
        test_target.append(target[t_id])        
        

print train_target.count('neutral') 

150


Классификация на три класса и матрица ошибок.

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])

pipeline.fit(train_texts, train_target)
predicted = pipeline.predict(test_texts)

print metrics.confusion_matrix(test_target, predicted)

[[34 11  6]
 [15 20 15]
 [ 9 12 29]]


Классификация на три класса и матрица ошибок.

In [38]:
train_idx = random.sample(range(0,201),  100) + random.sample(range(200, 401),  100)# + random.sample(range(400, 600),  150) 
train_texts = []
train_target = []
test_texts = []
test_target = []

for t_id in range(400):
    if t_id in train_idx:
        train_texts.append(texts[t_id])
        train_target.append(target[t_id])
    else:
        test_texts.append(texts[t_id])
        test_target.append(target[t_id])  
        
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])

pipeline.fit(train_texts, train_target)
predicted = pipeline.predict(test_texts)

print metrics.confusion_matrix(test_target, predicted)

[[79 21]
 [31 70]]


Классификация на два класса и матрица ошибок.